## Accuracy of QR factorisation methods

This notebook investigates the accuracy of various QR factorisation methods: GS, MGS and numpy's built in qr.

In [ ]:
# Function to set up a random matrix of size n x n with condition number 
# in the 2-norm equal to kappa.
# For more details, see the book "Accuracy and Stability of Numerical 
# Algorithms by Higham, section 28.3.
# This function uses the singular value decomposition of A.

import numpy as np

def randsvd(n, kappa):
    s = np.zeros(n)
    for i in range(n):
        beta = kappa**(1/(n-1))
        s[i] = beta**(-i)
    S = np.diag(s)
    
    def haar(n):
        A = np.random.randn(n, n)
        Q, R = np.linalg.qr(A)

        for i in range(n):
            if R[i, i] < 0:
                Q[:, i] *= -1
        return Q
    
    A = haar(n) @ S @ haar(n).T
    return A

For $n=100$ and $\kappa_2(A) = 10^k$, with $k=1, 2, \dots, 5$:

a) Set up a random matrix `A = randsvd(n, kappa)`.

b) Compute a QR factorisation of `A` using your `GS` function, your `MGS` function, and Numpy's `np.linalg.qr` function.

c) Are the computed matrices $\hat{Q}$ orthogonal? Do the computed factors $\hat{Q}$ and $\hat{R}$ satisfy $\hat{Q}\hat{R} = A$? Assess this by computing $\| \hat{Q}^T \hat{Q} - I \|_2$ and $\|\hat{Q}\hat{R} - A \|_2$.

d) Create a vector `xsol` to represent a vector $\bf x \in \mathbb{R}^n$, all of whose entries are $1$. Compute `b` such that $\bf b = Ax$. Use the three computed QR factorisations to solve $\bf Ax=b$. How do the errors $\| \bf \hat{x} - x \|_2$ between the computed solutions and the true solution compare?

In [ ]:
import numpy as np

def GS(A):
    n = A.shape[0]
    R = np.zeros((n, n))
    Q = np.zeros((n, n))
    
    for j in range(n):
        qhat = A[:, j].copy()
        
        for k in range(j):
            R[k, j] = Q[:, k].T @ A[:, j]
            qhat -= R[k, j] * Q[:, k]
        
        R[j, j] = np.linalg.norm(qhat, 2)
        Q[:, j] = qhat / R[j, j]

    return Q, R

def MGS(A):
    n = A.shape[0]
    R = np.zeros((n, n))
    Q = np.zeros((n, n))
    
    for j in range(n):
        qhat = A[:, j].copy()
        
        for k in range(j):
            R[k, j] = Q[:, k].T @ qhat
            qhat -= R[k, j] * Q[:, k]
        
        R[j, j] = np.linalg.norm(qhat, 2)
        Q[:, j] = qhat / R[j, j]
    
    return Q, R


# We need backsub to solve Rx = y
def backsub(U, y):
    n = U.shape[0]
    x = np.zeros(n)
    
    for j in range(n-1, -1, -1):
        x[j] = (y[j] - U[j, j+1:] @ x[j+1:]) / U[j, j]
    
    return x


# We write a convenience function to test the methods
def QR_tests(A, b, QR_fun, xsol):
    '''
    Compute the QR factorisation of A using the function QR_fun,
    solves Ax = b, computes and returns the required 2-norms.
    '''
    n = A.shape[0]
    Q, R = QR_fun(A)
    
    # Test Q, R matrices (c)
    e_QTQ = np.linalg.norm(Q.T @ Q - np.eye(n), 2)
    e_QR = np.linalg.norm(Q @ R - A, 2)
    
    # Compute y = Q^T b, solve Rx = y using backsub
    y = Q.T @ b
    x = backsub(R, y)
    
    # Compute the 2-norm of the error (d)
    e_x = np.linalg.norm(x - xsol, 2)
    
    return e_QTQ, e_QR, e_x

# Assess the 3 functions
n = 100
xsol = np.ones(n)
e_QTQ = []
e_QR = []
e_x = []

for k in range(1, 6):
    kappa = 10 ** k
    A = randsvd(n, kappa)
    b = A @ xsol
    
    e_QTQ_k = []
    e_QR_k = []
    e_x_k = []
    
    print('k = {:1d}   e_QTQ     e_QR      e_x'.format(k))
    
    for fun in [GS, MGS, np.linalg.qr]:
        err = QR_tests(A, b, fun, xsol)
        print('{:5s} {:9.3g} {:9.3g} {:9.3g}'.format(
            fun.__name__, err[0], err[1], err[2]))
        
        # Store the results for plotting
        e_QTQ_k.append(err[0])
        e_QR_k.append(err[1])
        e_x_k.append(err[2])
        
    e_QTQ.append(e_QTQ_k)
    e_QR.append(e_QR_k)
    e_x.append(e_x_k)
    print()
    
# Plot the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(12, 4))
lab = ['GS', 'MGS', 'qr']
titles = ['Error for Q^T Q', 'Error for QR', 'Solution error']
e = [e_QTQ, e_QR, e_x]

for i in range(3):
    kappa = [10 ** k for k in range(1, 6)]
    ax[i].plot(kappa, [n[0] for n in e[i]], label='GS')
    ax[i].plot(kappa, [n[1] for n in e[i]], label='MGS')
    ax[i].plot(kappa, [n[2] for n in e[i]], label='qr')
    ax[i].legend()
    ax[i].set_xlabel(r'$\kappa$')
    ax[i].set_title(titles[i])
    ax[i].set_xscale('log')
    ax[i].set_yscale('log')
plt.show();